In [75]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import numpy as np
import datetime

df=pd.read_excel(r'./DataSource/DailyUpdateForTG.xlsx',skiprows=1,sheet_name=1)
df['城市']=df['指标名称'].str.split(":").str[-1]
df=df.drop(columns=['指标名称','单位','来源','更新时间'])
df.to_hdf('Processed_Data/subway.h5',key='raw')
subway=pd.read_hdf('Processed_Data/subway.h5',key='raw')
subway=pd.melt(subway,id_vars=['城市'],var_name='date',value_name='subway')
subway=subway[subway['subway']>0]
subway['date'] = pd.to_datetime(subway['date']).dt.date
subway=subway.sort_values(by=['城市','date'],ascending=True)
subway['date']=subway['date'].astype('str')
subway['dt']=subway['date'].str[-5:]

subway['subway_last7d_mean']=subway.groupby('城市')['subway'].transform(lambda x: x.rolling(7, 2).mean())
subway['subway_WoW']=subway['subway_last7d_mean']/subway.groupby('城市')['subway_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=subway['date']>'2018-10-01'
c2=subway['date']<'2019-06-01'
sw_19=subway[c1&c2]
sw_19=sw_19[['城市', 'dt','subway_last7d_mean']]
sw_19['subway_last7d_mean']=sw_19.groupby('城市')['subway_last7d_mean'].transform(lambda x: x.shift(periods=-14)) #调整CNY
sw_19=sw_19.rename(columns={'subway_last7d_mean':'subway_19_rolling_mean'})

c2=subway['date'].str.contains('2022|2023')
sw_22=subway[c2]
sw_22=sw_22.merge(sw_19,how='left',on=['城市','dt'])
sw_22['subway_status']=sw_22['subway_last7d_mean']/sw_22['subway_19_rolling_mean']

subway.tail(2)

,城市,date,subway,dt,subway_last7d_mean,subway_WoW
39,长沙,2023-01-14,134.74,01-14,157.832857,0.162257
10,长沙,2023-01-15,119.95,01-15,155.222857,0.120847


In [76]:
# 2.2 WoW vs. GDP
riqi='2023-01-14'

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=sw_22.merge(city_popu,on='城市',how='left')

o=o[o['date']==riqi][['城市','GDP','subway_WoW']] 

o['size']=np.nan
o['group']='其他'

o=o.sort_values(by=['group','GDP'],ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-14


In [77]:
# 2.2 as of 2019 vs. GDP
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=sw_22.merge(city_popu,on='城市',how='left')

o=o[o['subway_status']<2][sw_22['date']==riqi]
o=o[['城市','GDP','subway_status']] 

o['size']=np.nan
o['group']='其他'

d=sw_22[sw_22['date']==riqi][['城市','date','subway_status']].reset_index(drop=True).rename(columns={'subway_status':'subway'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='subway')

o=o.sort_values(by=['group','GDP'],ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-14


In [70]:
#TY copy to wkbk
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','subway')
d1=pd.read_hdf('Processed_Data/tablehead.h5','subway')
wkbk=pd.pivot_table(subway[subway['date']>='2023-01-11'],index='城市',columns='date',values='subway',aggfunc='mean').reset_index()
d1.merge(wkbk,how='left',on='城市').drop(columns=['城市']).to_clipboard(index=False)

In [84]:
#总体恢复程度

first_tier='北京,上海,深圳,广州'.split(',')
second_tier='成都,西安,苏州,重庆,武汉,郑州,南京'.split(',')

sw_22.loc[sw_22['城市'].isin(first_tier),'citytier']='T1'
sw_22.loc[sw_22['城市'].isin(second_tier),'citytier']='T2'

bytier=sw_22.copy()
bytier['subway_last7d_mean']=bytier['subway_last7d_mean']*7
bytier['subway_19_rolling_mean']=bytier['subway_19_rolling_mean']*7
bytier=bytier.groupby(['citytier','date']).sum().reset_index()[['citytier','date','subway_last7d_mean','subway_19_rolling_mean']]
bytier['status']=bytier['subway_last7d_mean']/bytier['subway_19_rolling_mean']
bytier['status']=bytier['status'].transform(lambda x: x.rolling(7, 7).mean())
bytier=pd.pivot_table(bytier,index='date',columns='citytier',values='status',aggfunc='mean').reset_index()

alldata=sw_22.copy()
alldata['subway_last7d_mean']=alldata['subway_last7d_mean']*7
alldata['subway_19_rolling_mean']=alldata['subway_19_rolling_mean']*7
alldata=alldata[~alldata['citytier'].isnull()].groupby(['date']).sum().reset_index()[['date','subway_last7d_mean','subway_19_rolling_mean']]
alldata['status']=alldata['subway_last7d_mean']/alldata['subway_19_rolling_mean']
alldata['status']=alldata['status'].transform(lambda x: x.rolling(7, 7).mean())
alldata=alldata[['date','status']].rename(columns={'status':'all'})
alldata.tail(60).to_clipboard(index=False)